<a href="https://colab.research.google.com/github/MalihaUCF/Machine-Learning-Course-Assignments--Spring-2019/blob/master/Assignment1/Problem3/Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the requried libraries

from keras.datasets import mnist
import numpy as np
from keras import models
from keras import layers
from keras.utils import to_categorical 

Using TensorFlow backend.


In [2]:
#loading the MNIST dataset

(train_images,train_labels),(test_images,test_labels)=mnist.load_data()  

print('Training Data Shape',train_images.shape)
print('Test Data Shape',test_images.shape)
print('The train and test labels look like this,' ,train_labels.shape,test_labels.shape)

11493376/11490434 [==============================] - 0s 0us/step
Training Data Shape (60000, 28, 28)
Test Data Shape (10000, 28, 28)
The train and test labels look like this, (60000,) (10000,)


In [3]:
classes = np.unique(train_labels)
number_of_Classes = len(classes)
print('Total number of Classes : ', number_of_Classes)
print('Classes are : ', classes)

Total number of Classes :  10
Classes are :  [0 1 2 3 4 5 6 7 8 9]


In [4]:
#Defining the Single layered Network

model=models.Sequential()
model.add(layers.Dense(28*28,activation='relu',input_shape=(28*28,)))
model.add(layers.Dense(10,activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                7850      
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Preparing the Data for the network


train_images = train_images.reshape(60000, 28*28) 
test_images = test_images.reshape(10000, 28*28) 
train_images = train_images.astype('float32') 
test_images = test_images.astype('float32') 
train_images /= 255 
test_images /= 255


train_labels = to_categorical(train_labels,10)
test_labels = to_categorical(test_labels,10)

In [7]:
#Compiling the network

epochs=10
batch_size=128
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_images, train_labels, epochs=epochs,batch_size=batch_size, validation_data=(test_images, test_labels))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 111us/step - loss: 1.0707 - acc: 0.7743 - val_loss: 0.5796 - val_acc: 0.8703
Epoch 2/10
60000/60000 [==============================] - 6s 106us/step - loss: 0.5082 - acc: 0.8765 - val_loss: 0.4230 - val_acc: 0.8920
Epoch 3/10
60000/60000 [==============================] - 6s 106us/step - loss: 0.4106 - acc: 0.8931 - val_loss: 0.3653 - val_acc: 0.9042
Epoch 4/10
60000/60000 [==============================] - 6s 104us/step - loss: 0.3658 - acc: 0.9018 - val_loss: 0.3334 - val_acc: 0.9101
Epoch 5/10
60000/60000 [==============================] - 6s 103us/step - loss: 0.3382 - acc: 0.9080 - val_loss: 0.3122 - val_acc: 0.9158
Epoch 6/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.3184 - acc: 0.9122 - val_loss: 0.2968 - val_acc: 0.9186
Epoch 7/10
60000/60000 [==============================] - 6s 100us/step - 